In [5]:
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import plot_tree

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline
import seaborn as sns

import sys
sys.path.insert(1, '../src/stproject')
from utils import *

In [6]:
features_excluded = ['C#C', 'R3', 'R4', 'R7', 'R8', 'molecule']
X_train, X_valid, y_train, y_valid = load_data_fe0('../data/df_fe0.csv', features_excluded, 'measured_st')
X_train

,Ar,C,C=C,H,M,O-acid,O-alc,O-ald,O-ester,O-eth,O-ket,R5,R6,density
188,0.0,3.0,0.0,6.0,74.079,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.9342
183,0.0,5.0,0.0,10.0,102.133,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.8930
102,0.0,4.0,0.0,10.0,106.121,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.1800
185,0.0,6.0,0.0,12.0,116.160,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.8853
63,1.0,10.0,0.0,14.0,134.222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,0.0,9.0,0.0,20.0,128.259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7185
196,0.0,6.0,0.0,12.0,116.160,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.8660
74,0.0,4.0,0.0,10.0,74.123,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8096
111,0.0,6.0,0.0,14.0,150.174,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.1250


In [11]:
def score_xgb(params):
    print("Training with params : ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    # watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    model = xgb.train(params, dtrain, num_round)
    predictions = model.predict(dvalid)
    score = np.sqrt(mean_squared_error(y_valid, predictions))
    print("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}

def optimize_xgb(trials, score_func, algo=tpe.suggest, max_evals=250):

    # this function optimizes xgb parameters
    # trials = hyperopt trials object
    # score_xgb = score function
    # algo = algorithm use to suggest next point
    # returns best parameters

    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 1),
             'eta' : hp.quniform('eta', 0.01, 0.10, 0.02),
             'max_depth' : hp.choice('max_depth', np.arange(1, 13, 1)),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'eval_metric': 'rmse',
             'objective': 'reg:squarederror',
             'nthread' : 6,
             'silent' : 1,
             'seed': 42
             }

    best = fmin(score_func, space, algo=algo, trials=trials, max_evals=max_evals)
    print(best)


In [12]:
# Trials object where the history of search will be stored
trials = Trials()
optimize_xgb(trials, score_xgb, max_evals=5)

Training with params : 
{'n_estimators': <hyperopt.pyll.base.Apply object at 0x00000221A6450EC8>, 'eta': <hyperopt.pyll.base.Apply object at 0x00000221A6109B88>, 'max_depth': <hyperopt.pyll.base.Apply object at 0x00000221A645C0C8>, 'min_child_weight': <hyperopt.pyll.base.Apply object at 0x00000221A645CBC8>, 'subsample': <hyperopt.pyll.base.Apply object at 0x00000221A645CE88>, 'gamma': <hyperopt.pyll.base.Apply object at 0x00000221A6461C48>, 'colsample_bytree': <hyperopt.pyll.base.Apply object at 0x00000221A6461808>, 'eval_metric': 'rmse', 'objective': 'reg:squarederror', 'nthread': 6, 'silent': 1, 'seed': 42}


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Apply'

Best rmse of 1.421 achieved with colsample_by_tree: 0.55, eta: 0.06, gamma: 0.80, max_depth: 2, min_child_weight: 2.0, n_estimators: 102, subsample: 0.5

In [ ]:
# Training model with best params
xgb_best = xgb.XGBRegressor(n_estimators=102,
                            learning_rate=0.06,
                            max_depth=2,
                            min_child_weight=2,
                            subsample=0.5,
                            gamma=0.8,
                            colsample_by_tree=0.55,
                            objective='reg:squarederror',
                            n_jobs=6,
                            verbosity=3,
                           random_state=42)
xgb_best.fit(X_train, y_train, early_stopping_rounds=25, eval_set=[(X_valid, y_valid)])

In [ ]:
# feature importances
plt.barh(X_train.columns, xgb_best.feature_importances_)
plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(30, 30))
# plot_tree(xgb_best, ax=ax)

In [ ]:
print(f"rmse = {np.sqrt(np.mean((xgb_best.predict(X_valid)-y_valid)**2))}")

In [ ]:
pd.DataFrame({'y_hat': xgb_best.predict(X_valid), 'y': y_valid})

TODOs by priority:

0. try features scaled by molecular weight, molecular fragments only
1. do the same with multiple train-test(valid) splitting + weighted prediction
2. use triple split: train, valid, test with test set matching the literature data df_stliq_clean was taken from
3. test each final model against feature-engineered polymer

## XGB with train, valid, test split

In [ ]:
def load_data_fe0_2(features_excluded, target):
    df_ref = pd.read_csv('../data/df_stliq_clean.csv', index_col=0)
    df = pd.read_csv('../data/df_fe0.csv', index_col=0)
    df = df[df[features_excluded].sum(axis=1) == 0]
    
    # pulling index of test data from ref (original_id column contains 'Test')
    test_idx = df_ref[df_ref['original_id'].str.contains('Test')].index.tolist()
    df_train = df[~df.index.isin(test_idx)]
    df_test = df[df.index.isin(test_idx)]
    X_train = df_train[df_train.columns.difference(features_excluded + [target])]
    y_train = df_train[target]
    X_test = df_test[df_test.columns.difference(features_excluded + [target])]
    y_test = df_test[target]
    
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
X_train, X_valid, X_test, y_train, y_valid, y_test = \
load_data_fe0_2(features_excluded, 'measured_st')
len(X_train), len(X_valid), len(X_test)

In [ ]:
# trials2 = Trials()
# optimize(trials2)

Best rmse of 1.526 achieved with colsample_by_tree: 0.95, eta: 0.06, gamma: 0.5, max_depth: 6, min_child_weight: 4.0, n_estimators: 248, subsample: 0.85

In [ ]:
# check index consistency with 'append'
print(all(X_train.append(X_valid).index == y_train.append(y_valid).index))

In [ ]:
# Training model with best params
xgb_best2_1 = xgb.XGBRegressor(n_estimators=248,
                            learning_rate=0.06,
                            max_depth=6,
                            min_child_weight=4,
                            subsample=0.85,
                            gamma=0.5,
                            colsample_by_tree=0.95,
                            objective='reg:squarederror',
                            n_jobs=6,
                            verbosity=3,
                            random_state=42)
xgb_best2_1.fit(X_train.append(X_valid), y_train.append(y_valid), eval_set=[(X_test, y_test)])

In [ ]:
# feature importances
plt.barh(X_train.columns, xgb_best2_1.feature_importances_)
plt.show()

In [ ]:
print(f"rmse of XGB with fe0 + full features  = {np.sqrt(mean_squared_error(y_test, xgb_best2_1.predict(X_test)))}")
sns.scatterplot(xgb_best2_1.predict(X_test), y_test)
sns.lineplot(x=[17, 50], y=[15, 47], alpha=0.2)
plt.xlabel('predicted surface tension')
plt.ylabel('measured surface tension')
plt.title('XGB fe0 - full features', {'fontsize': 15, 'weight': 'bold'})

## Fragments only (scaled by molecular weight)

In [ ]:
def scale_M(df):
    # df contains all numeric features
    # this function drops 'density' feature and normalizes the molecular fragments by molecular weight
    return df[df.columns.difference(['density', 'M'])].divide(df['M'], axis=0)

In [ ]:
X_train = scale_M(X_train)
X_valid = scale_M(X_valid)
X_test = scale_M(X_test)

In [ ]:
trials3 = Trials()
optimize(trials3)

In [ ]:
# Training model with best params
xgb_best3 = xgb.XGBRegressor(n_estimators=472,
                            learning_rate=0.08,
                            max_depth=4,
                            min_child_weight=1,
                            subsample=0.5,
                            gamma=1,
                            colsample_by_tree=1,
                            objective='reg:squarederror',
                            n_jobs=6,
                            verbosity=3,
                            random_state=42)
xgb_best3.fit(X_train.append(X_valid), y_train.append(y_valid), early_stopping_rounds=25, eval_set=[(X_test, y_test)])

In [ ]:
# feature importances
plt.barh(X_train.columns, xgb_best3.feature_importances_)
plt.show()

In [ ]:
print(f"rmse of XGB with fe0 + features/M  = {np.sqrt(mean_squared_error(y_test, xgb_best3.predict(X_test)))}")
sns.scatterplot(xgb_best3.predict(X_test), y_test)
sns.lineplot(x=[17, 50], y=[15, 47], alpha=0.2)
plt.xlabel('predicted surface tension')
plt.ylabel('measured surface tension')
plt.title('XGB fe0 - features/M', {'fontsize': 15, 'weight': 'bold'})

## Fragments only (scaled by density)

In [ ]:
def scale_density(df):
    # df contains all numeric features
    # this function drops 'density' feature and normalizes the molecular fragments by molecular weight
    return df[df.columns.difference(['density', 'M'])].divide(df['density'], axis=0)

In [ ]:
X_train = scale_density(X_train)
X_valid = scale_density(X_valid)
X_test = scale_density(X_test)

In [ ]:
trials4 = Trials()
optimize(trials4)

# Preparation of test set

In [ ]:
df_test = pd.read_csv('../data/polyesters.csv')
df_diols_fe1 = pd.read_csv('../data/df_diols_fe1.csv', index_col=0)
df_acids_fe1 = pd.read_csv('../data/df_acids_fe1.csv', index_col=0)

# correcting 'O-alc', 'O-acid' and 'O-ester' by subtracting atom/functional group loss from condensation
df_diols_fe1['OH'] -= 2
df_acids_fe1['COOH'] -= 2
df_acids_fe1['COOR'] += 2

df_monomers_fe1 = pd.concat([df_diols_fe1, df_acids_fe1])
df_monomers_fe1

monomers = df_test.columns[df_test.columns.str.contains('diol|acid')]
features = df_monomers_fe1.columns

df_test_fe1 = avg_monomer_features(df_test, df_monomers_fe1, monomers, features)
st_test = df_test['measured_st']

### TEST 3

In [ ]:
X_test3 = (df_test_fe1[df_test_fe1.columns.difference(['M']+features_excluded)]
           .divide(df_test_fe1['M']-18, axis=0))
y_hat_test3 = xgb_best3.predict(X_test3)

In [ ]:
sns.scatterplot(y_hat_test3, st_test)

In [ ]:
print(f"rmse = {np.sqrt(np.mean((y_hat_test3-st_test)**2))}")